In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
embedding_matrix = torch.load(os.path.join(DATA_PATH, 'embedding_matrix.pt'))

In [4]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [5]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [6]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        y: a tensor of shape (# patiens, max # categories) of type torch.long
        y_masks: a tensor of shape (# patiens, max # categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]
#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    
#     return x, x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, y, y_masks

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [8]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [9]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    
    '''
    Arguments:
        train_dataset: train dataset of type `CustomDataset`
        test_dataset: test dataset of type `CustomDataset`
        val_dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def attention_sum(alpha, v, masks):
    """
    Arguments:
        alpha: the alpha attention weights of shape (batch_size, # visits, 1)
        v: the visit embeddings of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, embedding_dim)
    """
    masks = masks[:,:].any(dim=2)
    v_masked = v.clone()
    v_masked[~masks] = 0
    c = torch.sum(alpha * v_masked, dim=1)
    return c

In [11]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        Arguments:
            g: the output tensor from RNN-alpha of shape (batch_size, # visits, hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, # visits, 1)
        """
        m = nn.Softmax(dim=1)
        return m(self.a_att(g))

In [12]:
class EnhancedRNNa(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of higher level categories to predict
        """
        self.embedding = nn.Linear(num_codes, 300)
        self.embedding.weight.data = embedding_matrix
        self.rnn = nn.GRU(300, hidden_size=128, batch_first=True)
        self.att_a = AlphaAttention(128)
        self.fc = nn.Linear(300, num_categories)
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = indices_to_multihot(x, masks, 4903)
        x = self.embedding(x)
        x[~masks.any(dim=2)] = 0  # Set masked visits to 0
        output, _ = self.rnn(x)
        alpha = self.att_a(output)
        c = attention_sum(alpha, x, masks)
        logits = self.fc(c)        
        return logits

# load the model here
enhanced_rnna = EnhancedRNNa(num_codes = len(codes), num_categories=len(sub_categories))
enhanced_rnna

EnhancedRNNa(
  (embedding): Linear(in_features=4903, out_features=300, bias=True)
  (rnn): GRU(300, 128, batch_first=True)
  (att_a): AlphaAttention(
    (a_att): Linear(in_features=128, out_features=1, bias=True)
  )
  (fc): Linear(in_features=300, out_features=184, bias=True)
)

In [13]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(enhanced_rnna.parameters(), weight_decay=0.001)

In [14]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the EnhancedRNNa model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)
#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)

#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [15]:
def train(model, train_loader, val_loader, n_epochs):
    """
    Arguments:
        model: the EnhancedRNNa model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks)
#             y_mh = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, val_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")
    
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [16]:
def indices_to_multihot(indices, masks, dim):
    mh_size = tuple(indices.shape[:-1]) + (dim,)
    multihot = torch.zeros(mh_size, dtype=torch.float)
    if len(mh_size) == 3:
        for i, patient in enumerate(indices):
            for j, visit in enumerate(patient):
                if masks[i,j].sum() == 0:
                    break
                y_idx = visit[masks[i,j]].unique()
                multihot[i,j] = F.one_hot(y_idx.to(torch.int64), multihot.shape[-1]).sum(0)
    else:
        for idx, row in enumerate(indices):
            y_idx = row[masks[idx]].unique()
            multihot[idx] = F.one_hot(y_idx, dim).sum(0).float()
    return multihot

In [17]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [18]:
n_epochs = 100
%time train(enhanced_rnna, train_loader, val_loader, n_epochs)

CPU: 20.33
RAM %: 59.8
CPU: 20.55
RAM %: 59.8
Epoch: 1 	 Training Loss: 4.442405
Epoch: 1 	 Validation precision@k5: 0.4641, accuracy@k5: 0.2322
Epoch: 1 	 Validation precision@k10: 0.4671, accuracy@k10: 0.3895
Epoch: 1 	 Validation precision@k15: 0.5274, accuracy@k15: 0.5087
Epoch: 1 	 Validation precision@k20: 0.6018, accuracy@k20: 0.5995
Epoch: 1 	 Validation precision@k25: 0.6846, accuracy@k25: 0.6843
Epoch: 1 	 Validation precision@k30: 0.7505, accuracy@k30: 0.7505
CPU: 20.76
RAM %: 59.9
Epoch: 2 	 Training Loss: 4.232453
Epoch: 2 	 Validation precision@k5: 0.3905, accuracy@k5: 0.1966
Epoch: 2 	 Validation precision@k10: 0.3997, accuracy@k10: 0.3339
Epoch: 2 	 Validation precision@k15: 0.4633, accuracy@k15: 0.4471
Epoch: 2 	 Validation precision@k20: 0.5418, accuracy@k20: 0.5396
Epoch: 2 	 Validation precision@k25: 0.6362, accuracy@k25: 0.6360
Epoch: 2 	 Validation precision@k30: 0.7214, accuracy@k30: 0.7214
CPU: 20.85
RAM %: 60.1
Epoch: 3 	 Training Loss: 4.067099
Epoch: 3 	 Vali

CPU: 26.29
RAM %: 60.8
Epoch: 19 	 Training Loss: 3.584127
Epoch: 19 	 Validation precision@k5: 0.6288, accuracy@k5: 0.3312
Epoch: 19 	 Validation precision@k10: 0.5968, accuracy@k10: 0.5067
Epoch: 19 	 Validation precision@k15: 0.6414, accuracy@k15: 0.6211
Epoch: 19 	 Validation precision@k20: 0.7080, accuracy@k20: 0.7056
Epoch: 19 	 Validation precision@k25: 0.7669, accuracy@k25: 0.7667
Epoch: 19 	 Validation precision@k30: 0.8142, accuracy@k30: 0.8142
CPU: 26.59
RAM %: 60.7
Epoch: 20 	 Training Loss: 3.574200
Epoch: 20 	 Validation precision@k5: 0.6535, accuracy@k5: 0.3451
Epoch: 20 	 Validation precision@k10: 0.6025, accuracy@k10: 0.5120
Epoch: 20 	 Validation precision@k15: 0.6388, accuracy@k15: 0.6183
Epoch: 20 	 Validation precision@k20: 0.7043, accuracy@k20: 0.7019
Epoch: 20 	 Validation precision@k25: 0.7661, accuracy@k25: 0.7659
Epoch: 20 	 Validation precision@k30: 0.8125, accuracy@k30: 0.8125
CPU: 26.98
RAM %: 60.9
Epoch: 21 	 Training Loss: 3.561747
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8409, accuracy@k30: 0.8409
CPU: 29.46
RAM %: 61.1
Epoch: 37 	 Training Loss: 3.408292
Epoch: 37 	 Validation precision@k5: 0.7088, accuracy@k5: 0.3696
Epoch: 37 	 Validation precision@k10: 0.6624, accuracy@k10: 0.5601
Epoch: 37 	 Validation precision@k15: 0.6933, accuracy@k15: 0.6701
Epoch: 37 	 Validation precision@k20: 0.7472, accuracy@k20: 0.7444
Epoch: 37 	 Validation precision@k25: 0.7991, accuracy@k25: 0.7989
Epoch: 37 	 Validation precision@k30: 0.8389, accuracy@k30: 0.8389
CPU: 29.61
RAM %: 61.3
Epoch: 38 	 Training Loss: 3.400526
Epoch: 38 	 Validation precision@k5: 0.7106, accuracy@k5: 0.3722
Epoch: 38 	 Validation precision@k10: 0.6644, accuracy@k10: 0.5621
Epoch: 38 	 Validation precision@k15: 0.6915, accuracy@k15: 0.6685
Epoch: 38 	 Validation precision@k20: 0.7523, accuracy@k20: 0.7496
Epoch: 38 	 Validation precision@k25: 0.8037, accuracy@k25: 0.8034
Epoch: 38 	 Validation precision@k30: 0.8415, accuracy@k30: 0.8415
CPU: 29.82
RAM 

Epoch: 54 	 Validation precision@k25: 0.8067, accuracy@k25: 0.8064
Epoch: 54 	 Validation precision@k30: 0.8454, accuracy@k30: 0.8454
CPU: 26.54
RAM %: 61.3
Epoch: 55 	 Training Loss: 3.354577
Epoch: 55 	 Validation precision@k5: 0.7219, accuracy@k5: 0.3737
Epoch: 55 	 Validation precision@k10: 0.6731, accuracy@k10: 0.5667
Epoch: 55 	 Validation precision@k15: 0.7007, accuracy@k15: 0.6765
Epoch: 55 	 Validation precision@k20: 0.7563, accuracy@k20: 0.7534
Epoch: 55 	 Validation precision@k25: 0.8059, accuracy@k25: 0.8056
Epoch: 55 	 Validation precision@k30: 0.8435, accuracy@k30: 0.8435
CPU: 26.39
RAM %: 61.2
Epoch: 56 	 Training Loss: 3.354521
Epoch: 56 	 Validation precision@k5: 0.7157, accuracy@k5: 0.3712
Epoch: 56 	 Validation precision@k10: 0.6713, accuracy@k10: 0.5656
Epoch: 56 	 Validation precision@k15: 0.7020, accuracy@k15: 0.6777
Epoch: 56 	 Validation precision@k20: 0.7553, accuracy@k20: 0.7526
Epoch: 56 	 Validation precision@k25: 0.8046, accuracy@k25: 0.8043
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7522, accuracy@k20: 0.7493
Epoch: 72 	 Validation precision@k25: 0.8041, accuracy@k25: 0.8039
Epoch: 72 	 Validation precision@k30: 0.8425, accuracy@k30: 0.8425
CPU: 24.82
RAM %: 60.6
Epoch: 73 	 Training Loss: 3.343413
Epoch: 73 	 Validation precision@k5: 0.7165, accuracy@k5: 0.3698
Epoch: 73 	 Validation precision@k10: 0.6738, accuracy@k10: 0.5668
Epoch: 73 	 Validation precision@k15: 0.6998, accuracy@k15: 0.6753
Epoch: 73 	 Validation precision@k20: 0.7527, accuracy@k20: 0.7498
Epoch: 73 	 Validation precision@k25: 0.8045, accuracy@k25: 0.8042
Epoch: 73 	 Validation precision@k30: 0.8418, accuracy@k30: 0.8418
CPU: 24.63
RAM %: 61.0
Epoch: 74 	 Training Loss: 3.342831
Epoch: 74 	 Validation precision@k5: 0.7307, accuracy@k5: 0.3786
Epoch: 74 	 Validation precision@k10: 0.6745, accuracy@k10: 0.5673
Epoch: 74 	 Validation precision@k15: 0.7015, accuracy@k15: 0.6771
Epoch: 74 	 Validation precision@k20: 0.7556, accuracy@k20: 0.7527
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.7014, accuracy@k15: 0.6770
Epoch: 90 	 Validation precision@k20: 0.7561, accuracy@k20: 0.7531
Epoch: 90 	 Validation precision@k25: 0.8053, accuracy@k25: 0.8050
Epoch: 90 	 Validation precision@k30: 0.8422, accuracy@k30: 0.8422
CPU: 23.16
RAM %: 61.0
Epoch: 91 	 Training Loss: 3.338948
Epoch: 91 	 Validation precision@k5: 0.7181, accuracy@k5: 0.3717
Epoch: 91 	 Validation precision@k10: 0.6741, accuracy@k10: 0.5679
Epoch: 91 	 Validation precision@k15: 0.6988, accuracy@k15: 0.6744
Epoch: 91 	 Validation precision@k20: 0.7545, accuracy@k20: 0.7515
Epoch: 91 	 Validation precision@k25: 0.8047, accuracy@k25: 0.8044
Epoch: 91 	 Validation precision@k30: 0.8410, accuracy@k30: 0.8410
CPU: 23.01
RAM %: 61.2
Epoch: 92 	 Training Loss: 3.338646
Epoch: 92 	 Validation precision@k5: 0.7127, accuracy@k5: 0.3687
Epoch: 92 	 Validation precision@k10: 0.6683, accuracy@k10: 0.5631
Epoch: 92 	 Validation precision@k15: 0.7032, accuracy@k15: 0.6788
Epoch: 92 	 Val

In [19]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(enhanced_rnna, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7115, accuracy@k5: 0.3856
Validation precision@k10: 0.6742, accuracy@k10: 0.5775
Validation precision@k15: 0.7052, accuracy@k15: 0.6845
Validation precision@k20: 0.7601, accuracy@k20: 0.7572
Validation precision@k25: 0.8096, accuracy@k25: 0.8095
Validation precision@k30: 0.8451, accuracy@k30: 0.8451


In [20]:
torch.save(enhanced_rnna, os.path.join(CHECKPOINT_PATH, "EnhancedRNNa_100.pth"))